In [76]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [77]:
!unzip /content/gdrive/MyDrive/HotelRecommender/Hotel_Room_attributes.csv.zip

Archive:  /content/gdrive/MyDrive/HotelRecommender/Hotel_Room_attributes.csv.zip
replace Hotel_Room_attributes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Hotel_Room_attributes.csv  


In [78]:
!unzip /content/gdrive/MyDrive/HotelRecommender/Hotel_details.csv.zip

Archive:  /content/gdrive/MyDrive/HotelRecommender/Hotel_details.csv.zip
replace Hotel_details.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Hotel_details.csv       


In [79]:
!unzip /content/gdrive/MyDrive/HotelRecommender/hotel_price_min_maxFormula.zip

Archive:  /content/gdrive/MyDrive/HotelRecommender/hotel_price_min_maxFormula.zip
replace hotel_price_min_max - Formula.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: hotel_price_min_max - Formula.csv  


In [80]:
!unzip /content/gdrive/MyDrive/HotelRecommender/hotels_RoomPrice.csv.zip

Archive:  /content/gdrive/MyDrive/HotelRecommender/hotels_RoomPrice.csv.zip
replace hotels_RoomPrice.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: hotels_RoomPrice.csv    


In [81]:
!pip install tensorflow==2.1.0

In [82]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [83]:
pip install keras==2.3.1

In [84]:
import keras
print(keras.__version__)

2.3.1


In [85]:
!pip install pandas==0.25.3

In [86]:
import pandas as pd
print(pd.__version__)

0.25.3


In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from keras.callbacks import EarlyStopping
import math
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [89]:
hotel_details=pd.read_csv('/content/Hotel_details.csv',delimiter=',')
hotel_rooms=pd.read_csv('/content/Hotel_Room_attributes.csv',delimiter=',')
hotel_cost=pd.read_csv('/content/hotels_RoomPrice.csv',delimiter=',')

In [90]:
hotel_details.head()

,id,hotelid,hotelname,address,city,country,zipcode,propertytype,starrating,latitude,longitude,Source,url,curr
0,46406,1771651,Mediteran Bungalow Galeb,Vukovarska 7,Omis,Croatia,21310.0,Holiday parks,4,43.440124,16.682505,2,https://www.booking.com/hotel/hr/bungalow-luxu...,EUR
1,46407,177167,Hotel Polonia,Plac Teatralny 5,Torun,Poland,NaN,Hotels,3,53.012329,18.603800,5,https://www.agoda.com/en-gb/hotel-polonia/hote...,EUR
2,46408,1771675,Rifugio Sass Bece,"Belvedere del Pordoi,1",Canazei,Italy,38032.0,Hotels,3,46.477920,11.813350,2,http://www.booking.com/hotel/it/rifugio-sass-b...,EUR
3,46409,177168,Madalena Hotel,Mykonos,Mykonos,Greece,84600.0,Hotels,3,37.452316,25.329849,5,https://www.agoda.com/en-gb/madalena-hotel/hot...,EUR
4,46410,1771718,Pension Morenfeld,Mair im Korn Strasse 2,Lagundo,Italy,39022.0,Hotels,3,46.682780,11.131736,2,http://www.booking.com/hotel/it/pension-morenf...,EUR


In [91]:
hotel_rooms.head()

,id,hotelcode,roomamenities,roomtype,ratedescription
0,50677497,634876,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Double Room,"Room size: 15 m²/161 ft², Shower, 1 king bed"
1,50672149,8328096,Air conditioning: ;Closet: ;Fireplace: ;Free W...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
2,50643430,8323442,Air conditioning: ;Closet: ;Dishwasher: ;Firep...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
3,50650317,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"
4,50650318,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"


In [92]:
del hotel_details['id']
del hotel_rooms['id']
del hotel_details['zipcode']

In [93]:
hotel_details=hotel_details.dropna()
hotel_rooms=hotel_rooms.dropna()

In [94]:
hotel_details.drop_duplicates(subset='hotelid',keep=False,inplace=True)

In [95]:
hotel=pd.merge(hotel_rooms,hotel_details,left_on='hotelcode',right_on='hotelid',how='inner')

In [96]:
hotel.columns

Index(['hotelcode', 'roomamenities', 'roomtype', 'ratedescription', 'hotelid',
       'hotelname', 'address', 'city', 'country', 'propertytype', 'starrating',
       'latitude', 'longitude', 'Source', 'url', 'curr'],
      dtype='object')

In [97]:
del hotel['hotelid']
del hotel['url']
del hotel['curr']
del hotel['Source']

In [98]:
hotel.columns

Index(['hotelcode', 'roomamenities', 'roomtype', 'ratedescription',
       'hotelname', 'address', 'city', 'country', 'propertytype', 'starrating',
       'latitude', 'longitude'],
      dtype='object')

Recommender system based only on City and ratings about the hotel

In [99]:
def citybased(city):
    hotel['city']=hotel['city'].str.lower()
    citybase=hotel[hotel['city']==city.lower()]
    citybase=citybase.sort_values(by='starrating',ascending=False)
    citybase.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    if(citybase.empty==0):
        hname=citybase[['hotelname','starrating','address','roomamenities','ratedescription']]
        return hname.head()
    else:
        print('No Hotels Available')

In [100]:
print('Top 5 hotels')
citybased('London')

Top 5 hotels


,hotelname,starrating,address,roomamenities,ratedescription
48985,The Colonnade London Hotel,4,2 Warrington Crescent,Additional toilet: ;Air conditioning: ;Alarm c...,"Room size: 28 m²/301 ft², Balcony/terrace, Non..."
58663,South Point Suites - London Bridge,4,Bermondsey Street,Air conditioning: ;Alarm clock: ;Blackout curt...,"Room size: 24 m²/258 ft², Street view, Non-smo..."
106943,Doubletree by Hilton London Chelsea,4,Imperial Road Imperial Wharf,Air conditioning: ;Alarm clock: ;Blackout curt...,"Room size: 20 m²/215 ft², Street view, Non-smo..."
57452,Sir Christopher Wren Hotel & Spa,4,Unknown,Air conditioning: ;Alarm clock: ;Bathrobes: ;B...,"Room size: 25 m²/269 ft², Non-smoking, Shower ..."
92479,The Manor Hotel Heathrow,4,"Village Green, Datchet,",Additional bathroom: ;Additional toilet: ;Air ...,"Shower and bathtub, 1 double bed"


In [101]:
room_no=[
     ('king',2),
   ('queen',2), 
    ('triple',3),
    ('master',3),
   ('family',4),
   ('murphy',2),
   ('quad',4),
   ('double-double',4),
   ('mini',2),
   ('studio',1),
    ('junior',2),
   ('apartment',4),
    ('double',2),
   ('twin',2),
   ('double-twin',4),
   ('single',1),
     ('diabled',1),
   ('accessible',1),
    ('suite',2),
    ('one',2)
   ]

In [102]:
def calc():
    guests_no=[]
    for i in range(hotel.shape[0]):
        temp=hotel['roomtype'][i].lower().split()
        flag=0
        for j in range(len(temp)):
            for k in range(len(room_no)):
                if temp[j]==room_no[k][0]:
                    guests_no.append(room_no[k][1])
                    flag=1
                    break
            if flag==1:
                break
        if flag==0:
            guests_no.append(2)
    hotel['guests_no']=guests_no

calc()

In [103]:
def pop_citybased(city,number):
    hotel['city']=hotel['city'].str.lower()
    popbased=hotel[hotel['city']==city.lower()]
    popbased=popbased[popbased['guests_no']==number].sort_values(by='starrating',ascending=False)
    popbased.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    if popbased.empty==True:
        print('Sorry No Hotels Available\n tune your constraints')
    else:
        return popbased[['hotelname','roomtype','guests_no','starrating','address','roomamenities','ratedescription']].head(10)

In [104]:
pop_citybased('London',4)

,hotelname,roomtype,guests_no,starrating,address,roomamenities,ratedescription
1440,Holiday Inn London - Heathrow T5,Family Room,4,4,Old Bath Road,Air conditioning: ;Alarm clock: ;Blackout curt...,"Room size: 25 m²/269 ft², Non-smoking, Separat..."
53496,Mercure London Hyde Park Hotel,Family Room,4,4,8-14 Talbot Square,Air conditioning: ;Desk: ;Free Wi-Fi in all ro...,"Room size: 31 m²/334 ft², Non-smoking, 1 singl..."
60734,Hallmark Hotel London Chigwell Prince Regent,Executive Family Room,4,4,Manor Road,Air conditioning: ;Carpeting: ;Closet: ;Clothe...,"Non-smoking, Shower and bathtub, 1 double bed"
107246,Radisson Blu Edwardian Kenilworth - Bloomsbury,Family Room 2 adults + 2 children,4,4,97 Great Russell Street,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,"Room size: 20 m²/215 ft², Non-smoking, Shower ..."
55913,Acorn Lodge Gatwick,Family Stay & Park 8 Days,4,4,"79 Massetts Road, Horley",Air conditioning: ;Carpeting: ;Closet: ;Coffee...,"Shower, 1 double bed and 1 bunk bed"
64171,Gatwick Cambridge Hotel,Family Room,4,4,"19 Bonehurst Road, Horley",Air conditioning: ;Alarm clock: ;Carpeting: ;C...,"Room size: 15 m²/161 ft², Non-smoking, Shower ..."
96611,Trumbles Gatwick B&B,Family Room (2 Adults and 2 Children) - Number 6,4,4,Stan Hill,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,"Room size: 18 m²/194 ft², Shower and bathtub, ..."
97400,Heathrow/Windsor Marriott Hotel,"Deluxe Room, Guest room, 2 Double, Family Room",4,4,"Ditton Road, Langley",Air conditioning: ;Alarm clock: ;Carpeting: ;C...,"Room size: 30 m²/323 ft², Non-smoking, Shower ..."
65811,Atrium Hotel Heathrow,Family 1,4,4,Great south-west road Feltham,Additional toilet: ;Air conditioning: ;Alarm c...,"Room size: 28 m²/301 ft², Shower, 2 single bed..."
112259,The Level at Melia White House,The Level Family Room,4,4,Albany Street,Air conditioning: ;Bathrobes: ;Blackout curtai...,"Shower and bathtub, Executive lounge access, 1..."


In [105]:
hotel['roomamenities']=hotel['roomamenities'].str.replace(': ;',',')

In [106]:
def requirementbased(city,number,features):
    hotel['city']=hotel['city'].str.lower()
    hotel['roomamenities']=hotel['roomamenities'].str.lower()
    features=features.lower()
    features_tokens=word_tokenize(features)  
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    reqbased=hotel[hotel['city']==city.lower()]
    reqbased=reqbased[reqbased['guests_no']==number]
    reqbased=reqbased.set_index(np.arange(reqbased.shape[0]))
    l1 =[];l2 =[];cos=[];
    #print(reqbased['roomamenities'])
    for i in range(reqbased.shape[0]):
        temp_tokens=word_tokenize(reqbased['roomamenities'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        #print(rvector)
        cos.append(len(rvector))
    reqbased['similarity']=cos
    reqbased=reqbased.sort_values(by='similarity',ascending=False)
    reqbased.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    return reqbased[['hotelname','roomtype','guests_no','starrating','address','roomamenities','ratedescription','similarity']].head(10)

In [107]:
requirementbased('London',1,'I need a extra toilet and room should be completely air conditioned.I should have a bathrobe.')

,hotelname,roomtype,guests_no,starrating,address,roomamenities,ratedescription,similarity
310,The Colonnade London Hotel,Studio,1,4,2 Warrington Crescent,"additional toilet,air conditioning,alarm clock...","Room size: 28 m²/301 ft², Balcony/terrace, Non...",4
819,Amba Hotel Charing Cross,Studio King Suite,1,4,The Strand,"air conditioning,alarm clock,bathrobes,carpeti...","Room size: 34 m²/366 ft², Non-smoking, Shower ...",3
793,Holiday Inn Express Park Royal,2 Single Beds Non-smoking,1,3,Victoria RoadNorth Acton,"air conditioning,alarm clock,carpeting,closet,...","Room size: 25 m²/269 ft², Non-smoking, Shower,...",3
816,Simply Rooms & Suites Hotel,Deluxe Studio Suite,1,4,21 Avonmore Road,"additional toilet,air conditioning,cleaning pr...","Room size: 20 m²/215 ft², City view, Non-smoki...",3
912,The Cleveland Hotel,Single Room - Basic,1,4,39-40 Cleveland Square,"air conditioning,carpeting,closet,clothes rack...","Room size: 14 m²/151 ft², Non-smoking, Shower,...",3
408,South Point Suites - London Bridge,Deluxe Studio Suite,1,4,Bermondsey Street,"additional bathroom,additional toilet,air cond...","Room size: 45 m²/484 ft², 2 bathrooms, Shower,...",3
883,Angus Hotel,Single - Non-Smoking,1,2,31-32 Argyle square,"additional toilet,air conditioning,carpeting,c...","Street view, Non-smoking, Shower, 1 double bed",3
462,Pestana Chelsea Bridge Hotel And Spa,Accessible Twin Bedroom,1,4,354 Queenstown Road,"air conditioning,alarm clock,bathrobes,blackou...","Room size: 30 m²/323 ft², City view, Non-smoki...",3
494,Park Plaza London Waterloo,Studio King,1,4,6 Hercules Road,"air conditioning,alarm clock,bathrobes,blackou...","Room size: 24 m²/258 ft², City view, Non-smoki...",3
323,Park International Hotel,Standard Single Room,1,4,117-129 Cromwell Road,"additional toilet,air conditioning,blackout cu...","Room size: 11 m²/118 ft², Non-smoking, Shower,...",3


In [108]:
from math import sin, cos, sqrt, atan2, radians
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
secret_value_0 = '4519d076b432f5'
import requests
R=6373.0#Earth's Radius
sw = stopwords.words('english')
lemm = WordNetLemmatizer()
def hybrid(address,city,number,features):
    features=features.lower()
    features_tokens=word_tokenize(features)
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    data = {
    'key': secret_value_0,
    'q': address,
    'format': 'json'}
    response = requests.get(url, params=data)
    dist=[]
    lat1,long1=response.json()[0]['lat'],response.json()[0]['lon']
    lat1=radians(float(lat1))
    long1=radians(float(long1))
    hybridbase=hotel[hotel['guests_no']==number]
    hybridbase['city']=hybridbase['city'].str.lower()
    hybridbase=hybridbase[hybridbase['city']==city.lower()]
    hybridbase.drop_duplicates(subset='hotelcode',inplace=True,keep='first')
    hybridbase=hybridbase.set_index(np.arange(hybridbase.shape[0]))
    for i in range(hybridbase.shape[0]):
        lat2=radians(hybridbase['latitude'][i])
        long2=radians(hybridbase['longitude'][i])
        dlon = long2 - long1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        dist.append(distance)
    hybridbase['distance']=dist
    hybridbase=hybridbase.sort_values(by='distance',ascending=True)
    hybridbase=hybridbase.head(15)
    hybridbase=hybridbase.set_index(np.arange(hybridbase.shape[0]))
    coss=[]
    for i in range(hybridbase.shape[0]):
        temp_tokens=word_tokenize(hybridbase['roomamenities'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        coss.append(len(rvector))
    hybridbase['similarity']=coss
    return hybridbase.sort_values(by='similarity',ascending=False).head(10)

In [109]:
url = "https://us1.locationiq.com/v1/search.php"
hybrid("Big Ben,London",'London',4,'I need a extra toilet and room should be completely air conditioned.I should have a bathrobe.')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,hotelcode,roomamenities,roomtype,ratedescription,hotelname,address,city,country,propertytype,starrating,latitude,longitude,guests_no,distance,similarity
0,79139,"air conditioning,bathrobes,carpeting,closet,co...",St James Family King Room,"Room size: 35 m²/377 ft², Shower and bathtub, ...",The Trafalgar St. James,"2 Spring Gardens, Trafalgar Square",london,United Kingdom,Hotels,4,51.507397,-0.129185,4,0.811211,3
4,127330,"additional toilet,air conditioning,alarm clock...",Deluxe Family Suite,"Room size: 35 m²/377 ft², Non-smoking, 3 bathr...",Mornington Hotel London Victoria,25 Gloucester Street,london,United Kingdom,Hotels,4,51.490303,-0.140295,4,1.586945,3
14,175494,"air conditioning,alarm clock,bathrobes,blackou...",Deluxe Family Room,"Room size: 30 m²/323 ft², Non-smoking, Separat...",Pestana Chelsea Bridge Hotel And Spa,354 Queenstown Road,london,United Kingdom,Hotels,4,51.482426,-0.148745,4,2.631862,3
1,127425,"air conditioning,closet,coffee/tea maker,fan,f...",Quad Room,"Room size: 20 m²/215 ft², Shower, 2 single bed...",Melbourne House Hotel,79 Belgrave Road,london,United Kingdom,Hotels,3,51.490276,-0.137371,4,1.457966,2
2,84774,"air conditioning,coffee/tea maker,daily housek...",Standard Quad Room,"Room size: 20 m²/215 ft², Balcony/terrace, Non...",Huttons Hotel,53-57 Belgrave Road,london,United Kingdom,Hotels,3,51.490608,-0.138419,4,1.474883,2
3,127426,"air conditioning,alarm clock,carpeting,closet,...",Superior Family Room (3 Adults),"Room size: 17 m²/183 ft², Balcony/terrace, Non...",The Melita,35 Charlwood Street,london,United Kingdom,Hotels,3,51.490173,-0.138874,4,1.532155,2
5,466492,"air conditioning,coffee/tea maker,desk,free wi...",Quad Room,"Room size: 23 m²/248 ft², Non-smoking, Shower,...",St Georges Pimlico Hotel,115 St Georges Drive,london,United Kingdom,Hotels,3,51.488531,-0.139517,4,1.702462,2
7,148905,"air conditioning,closet,coffee/tea maker,daily...",Westminster Apartment,"Room size: 33 m²/355 ft², Street view, Non-smo...",Georgian House Hotel,37-39 St Georges Drive,london,United Kingdom,Hotels,4,51.490524,-0.143872,4,1.749995,2
8,84263,"air conditioning,blackout curtains,closet,coff...",Quad,"Non-smoking, Shower, 2 single beds and 1 doubl...",The Grapevine Hotel,117 Warwick Way,london,United Kingdom,Hotels,2,51.490773,-0.144941,4,1.789963,2
9,1134132,"air conditioning,closet,clothes rack,coffee/te...",Family Double Room,"Room size: 15 m²/161 ft², Shower, 1 bunk bed",The Z Hotel City,23-28 Fleet Street,london,United Kingdom,Hotels,4,51.514295,-0.107946,4,1.901806,2


In [110]:
hotel_cost.head()
#onsite rate is one important feature which could be useful to recommend
#we will drop the rest since it is present in other table and we are going to merge the 
hotel_cost=hotel_cost.drop(['id','refid','websitecode','dtcollected','ratedate','los','guests','roomtype','netrate','ratedescription','ratetype','sourceurl','roomamenities'
,'ispromo','closed','discount','promoname','status_code','taxstatus','taxtype','taxamount','proxyused','israteperstay','hotelblock','input_dtcollected'],axis=1)

In [111]:
hotel_cost.columns

Index(['hotelcode', 'onsiterate', 'currency', 'maxoccupancy',
       'mealinclusiontype'],
      dtype='object')

In [112]:
#To reccomend we are gonna check how much does the price vary from room to room if 
#the varience is small enough then it is better for them to recommend the hotel
hot=hotel_cost.groupby(['hotelcode','maxoccupancy'])

In [113]:
hotel_cost.sort_values(by=['onsiterate'],ascending=False)
hotel_cost=hotel_cost.drop_duplicates(subset=['hotelcode','maxoccupancy'],keep='first')

In [114]:
var=hot['onsiterate'].var().to_frame('varience')
l=[]
for i in range(hotel_cost.shape[0]):
    var1=var[var.index.get_level_values(0)==hotel_cost.iloc[i][0]]
    l.append(var1[var1.index.get_level_values(1)==hotel_cost.iloc[i][3]]['varience'][0])

In [115]:
hotel_cost['var']=l
hotel_cost=hotel_cost.fillna(0)
hotel_cost['mealinclusiontype']=hotel_cost['mealinclusiontype'].replace(0,'No Complimentary')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [116]:
hotel1=pd.merge(hotel,hotel_cost,left_on=['hotelcode','guests_no'],right_on=['hotelcode','maxoccupancy'],how='inner')

In [117]:
hotel1=hotel1.drop_duplicates(subset=['hotelcode','maxoccupancy'],keep='first')

In [118]:
hotel1.columns

Index(['hotelcode', 'roomamenities', 'roomtype', 'ratedescription',
       'hotelname', 'address', 'city', 'country', 'propertytype', 'starrating',
       'latitude', 'longitude', 'guests_no', 'onsiterate', 'currency',
       'maxoccupancy', 'mealinclusiontype', 'var'],
      dtype='object')

In [119]:
def pricing(address,city,number,features):
    h=hybrid(address,city,number,features)
    price_based=pd.merge(h,hotel_cost,left_on=['hotelcode','guests_no'],right_on=['hotelcode','maxoccupancy'],how='inner')
    del price_based['maxoccupancy']
    h=price_based.sort_values(by='var')
    return h.head()

In [120]:
pricing("Tower of London",'London',4,'I need an alarm clock and a kettle flask.')
#the description is just only for example.It portrays you can keep any specific request

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,hotelcode,roomamenities,roomtype,ratedescription,hotelname,address,city,country,propertytype,starrating,latitude,longitude,guests_no,distance,similarity,onsiterate,currency,mealinclusiontype,var
4,4998414,"air conditioning,closet,coffee/tea maker,desk,...",Executive Apartment,"Room size: 18 m²/194 ft², Non-smoking, Shower,...",Millenium Apartments,28-30 Rivington Street,london,United Kingdom,Hotels,4,51.525899,-0.081915,4,2.016404,0,170.13,USD,No Complimentary,0.000000
1,2314283,"air conditioning,alarm clock,coffee/tea maker,...",Family,"Room size: 20 m²/215 ft², Non-smoking, Shower,...",Comfort Inn St Pancras - Kings Cross,2-5 St. Chad's Street,london,United Kingdom,Hotels,3,51.529598,-0.122300,4,3.984763,2,175.39,USD,Free breakfast for {4},1054.239140
5,45042,"air conditioning,closet,coffee/tea maker,daily...",Two Bedroom Family Room,"Room size: 60 m²/646 ft², Lake view, Non-smoki...",DoubleTree by Hilton Hotel London - Docklands ...,265 Rotherhithe Street,london,United Kingdom,Hotels,4,51.504459,-0.033230,4,3.004195,0,329.45,USD,Free breakfast for {4},4216.886471
3,399328,"air conditioning,carpeting,closet,coffee/tea m...",Family,"Room size: 15 m²/161 ft², Street view, Non-smo...",Dockside Hotel,55 Sutton Street,london,United Kingdom,Hotels,3,51.512666,-0.053458,4,1.655028,0,0.00,USD,No Complimentary,5943.097200
0,789357,"air conditioning,alarm clock,carpeting,closet,...",Basic Family Room with Shared Bathroom,"Room size: 20 m²/215 ft², Non-smoking, Shared ...",Princess Hotel,35-39 Argyle Street,london,United Kingdom,Hotels,2,51.528744,-0.123086,4,3.972849,2,114.99,USD,No Complimentary,6209.714024


In [121]:
optimum_band=pd.read_csv('/content/hotel_price_min_max - Formula.csv')

In [122]:
from math import sin, cos, sqrt, atan2, radians
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
secret_value_0 = '4519d076b432f5'
import requests
R=6373.0#Earth's Radius
sw = stopwords.words('english')
lemm = WordNetLemmatizer()
def hybrid(address,city,number,features):
    features=features.lower()
    features_tokens=word_tokenize(features)
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    data = {
    'key': secret_value_0,
    'q': address,
    'format': 'json'}
    response = requests.get(url, params=data)
    dist=[]
    lat1,long1=response.json()[0]['lat'],response.json()[0]['lon']
    lat1=radians(float(lat1))
    long1=radians(float(long1))
    hybridbase=hotel
    #hybridbase=hotel[hotel['guests_no']==number]
    hybridbase['city']=hybridbase['city'].str.lower()
    hybridbase=hybridbase[hybridbase['city']==city.lower()]
    hybridbase.drop_duplicates(subset='hotelcode',inplace=True,keep='first')
    hybridbase=hybridbase.set_index(np.arange(hybridbase.shape[0]))
    for i in range(hybridbase.shape[0]):
        lat2=radians(hybridbase['latitude'][i])
        long2=radians(hybridbase['longitude'][i])
        dlon = long2 - long1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        dist.append(distance)
    hybridbase['distance']=dist
    hybridbase=hybridbase[hybridbase['distance']<=5]
    hybridbase=hybridbase.set_index(np.arange(hybridbase.shape[0]))
    coss=[]
    for i in range(hybridbase.shape[0]):
        temp_tokens=word_tokenize(hybridbase['roomamenities'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        coss.append(len(rvector))
    hybridbase['similarity']=coss
    return hybridbase.sort_values(by='similarity',ascending=False)

def price_band_based(address,city,number,features):
    h=hybrid(address,city,number,features)
    price_band=pd.merge(h,optimum_band,left_on=['hotelcode'],right_on=['hotelcode'],how='inner')
    price_band=pd.merge(price_band,hotel_cost,left_on=['hotelcode'],right_on=['hotelcode'],how='inner')
    del price_band['min']
    del price_band['max']
    del price_band['Diff_Min']
    del price_band['Diff_Max']
    del price_band['currency']
    del price_band['country']
    del price_band['propertytype']
    del price_band['starrating']
    del price_band['latitude']
    del price_band['longitude']
    del price_band['guests_no']
    del price_band['var']
    price_band=price_band[price_band['Score']<=0.5]
    price_band=price_band[price_band['maxoccupancy']>=number]
    return price_band

In [123]:
price_band_based("Tower of London",'London',4,'I need an alarm clock and a kettle flask.').head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,hotelcode,roomamenities,roomtype,ratedescription,hotelname,address,city,distance,similarity,Score,onsiterate,maxoccupancy,mealinclusiontype
12,789357,"air conditioning,alarm clock,carpeting,closet,...",Standard Twin Room,"Room size: 15 m²/161 ft², Non-smoking, Shower,...",Princess Hotel,35-39 Argyle Street,london,3.972849,2,0.16,114.99,4,No Complimentary
17,2314283,"air conditioning,alarm clock,coffee/tea maker,...",Single,"Room size: 20 m²/215 ft², Non-smoking, Shower,...",Comfort Inn St Pancras - Kings Cross,2-5 St. Chad's Street,london,3.984763,2,0.14,175.39,4,Free breakfast for {4}
23,4453863,"air conditioning,alarm clock,blackout curtains...",Studio Suite,"Room size: 24 m²/258 ft², Street view, Non-smo...",South Point Suites - London Bridge,Bermondsey Street,london,1.110365,2,0.26,187.95,4,No Complimentary
38,148900,"air conditioning,alarm clock,carpeting,closet,...",Standard Single,"City view, Non-smoking, Shared bathroom, 1 sin...",Central Hotel London - King's Cross,"16 - 18 Argyle Street, Kings Cross",london,4.045245,2,0.10,163.67,4,Free breakfast for {4}
87,84774,"air conditioning,coffee/tea maker,daily housek...",Standard Quad Room,"Room size: 20 m²/215 ft², Balcony/terrace, Non...",Huttons Hotel,53-57 Belgrave Road,london,4.726689,0,0.16,0.00,4,No Complimentary


In [125]:
import pickle
Pkl_Filename = "Pickle_RL_Model.pkl"  
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(price_band_based, file)